In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

df = pd.read_json('yelp_academic_dataset_business.json', lines = True)
restaurant_df = df[df['categories'].str.contains('Restaurants', na = False)]
restaurant_df = restaurant_df.reset_index(drop = True)
restaurant_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '..."
2,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', '...","Pubs, Restaurants, Italian, Bars, American (Tr...",None
3,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '..."
4,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52263,l9eLGG9ZKpLJzboZq-9LRQ,Wawa,19 N Bishop Ave,Clifton Heights,PA,19018,39.925656,-75.310344,3.0,11,1,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Restaurants, Sandwiches, Convenience Stores, C...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
52264,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,1181 N Milwaukee St,Boise,ID,83704,43.615401,-116.284689,4.0,33,1,"{'WiFi': ''free'', 'RestaurantsGoodForGroups':...","Cafes, Juice Bars & Smoothies, Coffee & Tea, R...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-17:0', '..."
52265,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,1108 S 9th St,Philadelphia,PA,19147,39.935982,-75.158665,4.5,35,1,"{'WheelchairAccessible': 'False', 'Restaurants...","Restaurants, Mexican","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
52266,2O2K6SXPWv56amqxCECd4w,The Plum Pit,4405 Pennell Rd,Aston,DE,19014,39.856185,-75.427725,4.5,14,1,"{'RestaurantsDelivery': 'False', 'BusinessAcce...","Restaurants, Comfort Food, Food, Food Trucks, ...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."


In [2]:
def get_categories(row_categories, current_categories):    
    for category in row_categories:
        if category.strip() not in current_categories and category.strip() != 'Restaurants':

            current_categories.add(category.strip())
    return current_categories

all_rows_data = []
all_categories = set()

for i in tqdm(range(0, len(restaurant_df))):
    # Get the row data
    temp_row = restaurant_df.iloc[i]
    row_categories = temp_row['categories']
    if row_categories is None:
        continue
    row_categories = row_categories.split(',')
    row_categories = set([category.strip() for category in row_categories])
    
    # Add new categories to the current_categories set
    all_categories = get_categories(row_categories, all_categories)
  
    # Create a dictionary to store the row data
    row_data = {}
    
    for category in all_categories:
        result = 1 if category in row_categories else 0
        row_data[category] = result
    row_data['business_id'] = temp_row['business_id']
    all_rows_data.append(row_data)

exploded_df = pd.DataFrame(all_rows_data)
exploded_category_df = exploded_df.fillna(0)
df_without_business_id = exploded_category_df.drop(columns=["business_id"])
column_sums = df_without_business_id.sum()
columns_to_drop = column_sums[column_sums < 1000].index
exploded_category_df = exploded_category_df.drop(columns_to_drop, axis=1)
exploded_category_df

100%|██████████| 52268/52268 [00:04<00:00, 11067.11it/s]


,Coffee & Tea,Food,Bakeries,business_id,Burgers,Sandwiches,Fast Food,Ice Cream & Frozen Yogurt,Bars,American (Traditional),...,Sports Bars,Juice Bars & Smoothies,Mexican,Mediterranean,Wine & Spirits,Beer,Barbeque,Vegetarian,Desserts,Tacos
0,1,1,1,MTSW4McQd7CbVtyjqoe9mw,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,0,CF33F8-E6oudUQ46HnavjQ,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,k0hlBqXX-Bt0vf1op7Jr1w,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1,0,bBDDEgkFA1Otx9Lfe7BZUQ,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,1,0,eEOYSgkmpB90uNA7lDOMRA,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52263,1,1,0,l9eLGG9ZKpLJzboZq-9LRQ,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52264,1,1,0,cM6V90ExQD6KMSU3rRB5ZA,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52265,0,0,0,WnT9NIzQgLlILjPT0kEcsQ,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52266,0,1,0,2O2K6SXPWv56amqxCECd4w,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# Simple attributes
simple_attributes = ['OutdoorSeating', "CoatCheck", "Caters", "HappyHour", "DriveThru", "RestaurantsPriceRange2", "Alcohol", "RestaurantsAttire", "RestaurantsReservations", "Caters", "RestaurantsTakeOut", "RestaurantsGoodForGroups", "HappyHour", "DriveThru"]

# Create a list of dictionaries
data = []
for index, row in restaurant_df.iterrows():
    attributes = row['attributes']

    if attributes is None:
        continue

    row_data = {}
    row_data['business_id'] = row['business_id']  # Include the business_id of the row

    for attribute in simple_attributes:
        if attribute in attributes:
            value = attributes[attribute]
            if value is None:
                value = 'nan'  # or 'nil'
            row_data[attribute] = value
        else:
            row_data[attribute] = 'nan'  # or 'nil'
    data.append(row_data)

# Create a pandas DataFrame
df_simple_attributes = pd.DataFrame(data)
df_simple_attributes

,business_id,OutdoorSeating,CoatCheck,Caters,HappyHour,DriveThru,RestaurantsPriceRange2,Alcohol,RestaurantsAttire,RestaurantsReservations,RestaurantsTakeOut,RestaurantsGoodForGroups
0,MTSW4McQd7CbVtyjqoe9mw,False,nan,True,nan,nan,1,u'none',nan,nan,True,nan
1,CF33F8-E6oudUQ46HnavjQ,True,False,False,False,True,1,u'none',u'casual',False,True,True
2,k0hlBqXX-Bt0vf1op7Jr1w,True,nan,True,nan,nan,1,u'full_bar',u'casual',False,True,True
3,bBDDEgkFA1Otx9Lfe7BZUQ,True,False,False,False,True,1,u'none','casual',False,True,False
4,eEOYSgkmpB90uNA7lDOMRA,None,nan,nan,nan,nan,nan,'none',nan,False,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...
51698,l9eLGG9ZKpLJzboZq-9LRQ,nan,nan,True,nan,nan,1,nan,nan,nan,True,nan
51699,cM6V90ExQD6KMSU3rRB5ZA,True,nan,False,nan,nan,1,u'none','casual',False,False,True
51700,WnT9NIzQgLlILjPT0kEcsQ,False,nan,True,False,nan,2,u'none','casual',False,True,True
51701,2O2K6SXPWv56amqxCECd4w,nan,nan,True,nan,nan,1,'none',nan,nan,True,nan


In [4]:
merged_df = pd.merge(exploded_category_df, df_simple_attributes, on = 'business_id')
merged_df = pd.merge(merged_df, restaurant_df[['business_id', 'stars', 'review_count']], on = 'business_id')
# merged_df = merged_df.drop('business_id', axis=1)
merged_df = pd.get_dummies(merged_df, columns=['Alcohol', 'RestaurantsAttire'], dummy_na=True)
merged_df.to_csv('merged_data.csv', index=False)
merged_df

,Coffee & Tea,Food,Bakeries,business_id,Burgers,Sandwiches,Fast Food,Ice Cream & Frozen Yogurt,Bars,American (Traditional),...,Alcohol_nan,RestaurantsAttire_'casual',RestaurantsAttire_'dressy',RestaurantsAttire_'formal',RestaurantsAttire_None,RestaurantsAttire_nan,RestaurantsAttire_u'casual',RestaurantsAttire_u'dressy',RestaurantsAttire_u'formal',RestaurantsAttire_nan
0,1,1,1,MTSW4McQd7CbVtyjqoe9mw,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,True,False,False,False,False
1,0,1,0,CF33F8-E6oudUQ46HnavjQ,1.0,1.0,1.0,1.0,0.0,0.0,...,False,False,False,False,False,False,True,False,False,False
2,0,0,0,k0hlBqXX-Bt0vf1op7Jr1w,0.0,0.0,0.0,0.0,1.0,1.0,...,False,False,False,False,False,False,True,False,False,False
3,0,1,0,bBDDEgkFA1Otx9Lfe7BZUQ,1.0,0.0,1.0,1.0,0.0,0.0,...,False,True,False,False,False,False,False,False,False,False
4,0,1,0,eEOYSgkmpB90uNA7lDOMRA,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51698,1,1,0,l9eLGG9ZKpLJzboZq-9LRQ,0.0,1.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,True,False,False,False,False
51699,1,1,0,cM6V90ExQD6KMSU3rRB5ZA,0.0,0.0,0.0,0.0,0.0,0.0,...,False,True,False,False,False,False,False,False,False,False
51700,0,0,0,WnT9NIzQgLlILjPT0kEcsQ,0.0,0.0,0.0,0.0,0.0,0.0,...,False,True,False,False,False,False,False,False,False,False
51701,0,1,0,2O2K6SXPWv56amqxCECd4w,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,True,False,False,False,False
